In [1]:
import os
from clean_data import CleanTabular, CleanImage
import numpy as np
from PIL import Image
from pytorch import FbDataset, CNN, Resnet50, train, accuracy
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
from sklearn.model_selection import train_test_split

In [ ]:
image_path = os.path.join(os.path.dirname(os.getcwd()), 'image_data', 'fb_image')
image_cleaner = CleanImage(image_path)
cleaned_img_folder = image_cleaner.process_images(resized_pixel=(128, 128), mode = 'RGB')

In [2]:
parent_path = os.path.dirname(os.getcwd())
cleaned_img_folder = os.path.join(parent_path, "cleaned_image_data")
cleaned_image_paths = []
for dirpaths, dirnames, filenames in os.walk(cleaned_img_folder):
    for filename in filenames:
        cleaned_image_paths.append(os.path.join(cleaned_img_folder, filename))


In [3]:
tabular_cleaner = CleanTabular(os.path.join(os.path.dirname(os.getcwd()), 'data/Images.csv'))
tabular_cleaner.slice_df(['id', 'product_id'])
image_df = tabular_cleaner.df
image_df['image_data'] = np.NaN
image_df['image_data']=image_df['image_data'].astype('object')

convert_tensor = transforms.ToTensor()

for i in range(len(image_df['id'])):
    img = Image.open(os.path.join(cleaned_img_folder, image_df['id'][i] + '.jpg'))
    tensor = convert_tensor(img)
    numpydata = tensor.numpy()
    image_df.at[i, 'image_data'] = numpydata

In [4]:
tabular_cleaner = CleanTabular(os.path.join(os.path.dirname(os.getcwd()), 'data/Products.csv'))
tabular_cleaner.slice_df(['id', 'category'])
tabular_cleaner.clean_to_general_category('category')
cat_code_mapper = tabular_cleaner.clean_to_category_type('category', ohe=False)
product_df = tabular_cleaner.df
product_df = product_df.rename(columns={'id':'product_id'})

In [5]:
merged_df = image_df.merge(product_df, how='left', on='product_id')
merged_df = merged_df.drop(['product_id'], axis=1)
merged_df = merged_df.drop(['id'], axis=1)
# merged_df.to_csv('merged_df.csv', index = False)

In [18]:
merged_df['category'].unique()

array([ 6,  1,  2,  9, 11,  4,  0,  7,  3, 10,  5,  8, 12], dtype=int8)

In [6]:
fb_dataset = FbDataset(merged_df)

In [7]:
train_set, discard = torch.utils.data.random_split(fb_dataset, [1000, 11604])
train_set, test_set = torch.utils.data.random_split(train_set, [800, 200])

In [8]:
train_loader = DataLoader(train_set, shuffle = True, batch_size = 16)

In [ ]:
model = CNN(3, 565504)
train(model, train_loader, 0.001, epochs = 5)

In [9]:
model_resnet50 = Resnet50()
accuracy(model_resnet50, train_set)
print('val ended')
train(model_resnet50, train_loader, 0.03, epochs = 10, model_name='resnet50_200dataaset')
accuracy(model_resnet50, train_set)
print('val ended')

Using cache found in /Users/serenawong/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/Users/serenawong/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/Users/serenawong/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(


0.08875
val ended
0.2175
val ended


In [10]:
new_model = Resnet50()
state_dict = torch.load(os.path.join('model_evaluation', 'resnet50_200dataaset','weights', "2022-10-02 19:22:31.827969.pt"))
new_model.load_state_dict(state_dict)
accuracy(new_model, train_set)

Using cache found in /Users/serenawong/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


0.11125


0.11125